In [1]:
import numpy as np
from collections import deque
from copy import deepcopy

In [2]:
test0 = ['.....',
         '.S-7.',
         '.|.|.',
         '.L-J.',
         '.....']

test1 = ['..F7.',
         '.FJ|.',
         'SJ.L7',
         '|F--J',
         'LJ...']

data = np.genfromtxt('day10_input.txt', dtype=str, delimiter='\n', comments=None)

In [3]:
def get_graph(data):
    new_data = []
    for line in data:
        new_data.append(list(line))
        
    new_data = np.array(new_data)
    start = np.where(new_data == 'S')
    
    return new_data, (start[0][0], start[1][0])

#Check if position is traversable
def good_pos_part1(graph, pos, cur_val, dxdy):
    x = pos[0]
    y = pos[1]
    if x < 0 or x >= len(graph):
        return False
    if y < 0 or y >= len(graph[x]):
        return False
    
    mapping = {'|': {(-1,0): ['|', '7', 'F'], (1,0):  ['|', 'L', 'J']},
               '-': {(0,-1): ['-', 'L', 'F'], (0,1):  ['-', 'J', '7']},
               'L': {(-1,0): ['|', '7', 'F'], (0,1):  ['-', 'J', '7']},
               'J': {(-1,0): ['|', '7', 'F'], (0,-1): ['-', 'L', 'F']},
               '7': {(1,0):  ['|', 'L', 'J'], (0,-1): ['-', 'L', 'F']},
               'F': {(1,0):  ['|', 'L', 'J'], (0,1):  ['-', 'J', '7']},
               'S': {(-1,0): ['|', '7', 'F'], (1,0):  ['|', 'L', 'J'],
                     (0,-1):  ['-', 'L', 'F'], (0,1): ['-', 'J', '7']}}
    if graph[x][y] not in mapping.keys():
        return False
    if graph[x][y] not in mapping[cur_val][tuple(dxdy)]:
        return False
               
    return True

#Breadth-First Search
def BFS_part1(graph, start):
    
    dist_graph = np.zeros(graph.shape, dtype=int)
    dist_graph[start[0],start[1]] = 9
    next_graph = np.empty([graph.shape[0]+2,graph.shape[1]+2], dtype=str)
    for i in range(0, len(next_graph)):
        for j in range(0, len(next_graph[i])):
            next_graph[i,j] = '.'
    next_graph[start[0]+1,start[1]+1] = 'S'
    
    dxdy = {'|': [[-1,0],[1,0]],
            '-': [[0,-1],[0,1]],
            'L': [[-1,0],[0,1]],
            'J': [[-1,0],[0,-1]],
            '7': [ [1,0],[0,-1]],
            'F': [ [1,0],[0,1]],
            'S': [[-1,0],[1,0],[0,-1],[0,1]]}
    
    queue = deque([start])
    dist = {start: 0}
    
    while len(queue):
        cur_pos = queue.popleft()
        cur_dist = dist[cur_pos]
        cur_val = graph[cur_pos[0]][cur_pos[1]]
        
        #print(cur_pos, cur_dist, cur_val)
        
        for xy in dxdy[cur_val]:
            nxt_dist = cur_dist + 1
            nxt_pos = (cur_pos[0]+xy[0], cur_pos[1]+xy[1])
            #print('\t', xy, graph[nxt_pos[0],nxt_pos[1]])
            
            if good_pos_part1(graph, nxt_pos, cur_val, xy) and nxt_pos not in dist.keys():
                queue.append(nxt_pos)
                dist[nxt_pos] = nxt_dist
                dist_graph[nxt_pos[0],nxt_pos[1]] = nxt_dist
                next_graph[nxt_pos[0]+1,nxt_pos[1]+1] = graph[nxt_pos[0],nxt_pos[1]]
                
            elif nxt_pos in dist.keys() and dist[nxt_pos] == nxt_dist:
                #for line in dist_graph:
                #    print(line)
                #for line in next_graph:
                #    print(line)
                start = (start[0]+1, start[1]+1)
                
                #top
                if next_graph[start[0]-1, start[1]] == '|' or next_graph[start[0]-1, start[1]] == '7' or\
                   next_graph[start[0]-1, start[1]] == 'F':
                    #bottom
                    if next_graph[start[0]+1, start[1]] == '|' or\
                       next_graph[start[0]+1, start[1]] == 'J' or\
                       next_graph[start[0]+1, start[1]] == 'L':
                        next_graph[start[0], start[1]] = '|'
                    #right
                    elif next_graph[start[0], start[1]+1] == '-' or\
                         next_graph[start[0], start[1]+1] == 'J' or\
                         next_graph[start[0], start[1]+1] == '7':
                        next_graph[start[0], start[1]] = 'L'
                    #left
                    elif next_graph[start[0], start[1]-1] == '-' or\
                         next_graph[start[0], start[1]-1] == 'L' or\
                         next_graph[start[0], start[1]-1] == 'F':
                        next_graph[start[0], start[1]] = 'J'
                        
                #bottom
                elif next_graph[start[0]+1, start[1]] == '|' or next_graph[start[0]+1, start[1]] == 'L' or\
                   next_graph[start[0]+1, start[1]] == 'J':
                    #right
                    if next_graph[start[0], start[1]+1] == '-' or\
                       next_graph[start[0], start[1]+1] == 'J' or\
                       next_graph[start[0], start[1]+1] == '7':
                        next_graph[start[0], start[1]] = 'F'
                    #left
                    elif next_graph[start[0], start[1]-1] == '-' or\
                         next_graph[start[0], start[1]-1] == 'L' or\
                         next_graph[start[0], start[1]-1] == 'F':
                        next_graph[start[0], start[1]] = '7'
                        
                #left 
                elif next_graph[start[0], start[1]-1] == '-' or next_graph[start[0], start[1]-1] == 'L' or\
                   next_graph[start[0], start[1]-1] == 'F':
                    #right
                    if next_graph[start[0], start[1]+1] == '-' or\
                       next_graph[start[0], start[1]+1] == 'J' or\
                       next_graph[start[0], start[1]+1] == '7':
                        next_graph[start[0], start[1]] = '-'
                
                return nxt_dist, next_graph
    return -1

graph0, start0 = get_graph(test0)
dist0, ngraph0 = BFS_part1(graph0, start0)
print(dist0)
graph1, start1 = get_graph(test1)
dist1, ngraph1 = BFS_part1(graph1, start1)
print(dist1)
data_graph, data_start = get_graph(data)
data_dist, data_ngraph = BFS_part1(data_graph, data_start)
print('Part 1 result:', data_dist)

4
8
Part 1 result: 6846


In [4]:
test2 = ['...........',
         '.S-------7.',
         '.|F-----7|.',
         '.||.....||.',
         '.||.....||.',
         '.|L-7.F-J|.',
         '.|..|.|..|.',
         '.L--J.L--J.',
         '...........']

test3 = ['..........',
         '.S------7.',
         '.|F----7|.',
         '.||....||.',
         '.||....||.',
         '.|L-7F-J|.',
         '.|..||..|.',
         '.L--JL--J.',
         '..........']

test4 = ['.F----7F7F7F7F-7....',
         '.|F--7||||||||FJ....',
         '.||.FJ||||||||L7....',
         'FJL7L7LJLJ||LJ.L-7..',
         'L--J.L7...LJS7F-7L7.',
         '....F-J..F7FJ|L7L7L7',
         '....L7.F7||L7|.L7L7|',
         '.....|FJLJ|FJ|F7|.LJ',
         '....FJL-7.||.||||...',
         '....L---J.LJ.LJLJ...']

test5 = ['FF7FSF7F7F7F7F7F---7',
         'L|LJ||||||||||||F--J',
         'FL-7LJLJ||||||LJL-77',
         'F--JF--7||LJLJ7F7FJ-',
         'L---JF-JLJ.||-FJLJJ7',
         '|F|F-JF---7F7-L7L|7|',
         '|FFJF7L7F-JF7|JL---7',
         '7-L-JL7||F7|L7F-7F7|',
         'L.L7LFJ|||||FJL7||LJ',
         'L7JLJL-JLJLJL--JLJ.L']

In [5]:
def get_next_pos(graph, pos, dxdy, os=0):
    #os tracks edges
    
    xy = deepcopy(dxdy)
    
    x = pos[0]+xy[0]
    y = pos[1]+xy[1]
    
    while True:
        if x < 0 or x >= len(graph):
            return [None]
        if y < 0 or y >= len(graph[x]):
            return [None]

        if graph[x,y] == '.':
            return [(x,y)]
        
        if graph[x,y] == '-' and (xy[0] == -1 or xy[0] == 1):
            if xy[0] == -1:
                xy[0] = 0
                xy[1] = 1
                pos1 = get_next_pos(graph, (x,y), xy, 1)
                xy[1] = -1
                pos2 = get_next_pos(graph, (x,y), xy, 1)
            else: #xy[0] == 1
                xy[0] = 0
                xy[1] = 1
                pos1 = get_next_pos(graph, (x,y), xy, -1)
                xy[1] = -1
                pos2 = get_next_pos(graph, (x,y), xy, -1)
            
            ret_pos = []
            for p in pos1:
                if p is not None:
                    ret_pos.append(p)
            for p in pos2:
                if p is not None:
                    ret_pos.append(p)
            if len(ret_pos) > 0:
                return ret_pos
            return [None]
        
        if graph[x,y] == '|' and (xy[1] == -1 or xy[1] == 1):
            if xy[1] == -1:
                xy[0] = 1
                xy[1] = 0
                pos1 = get_next_pos(graph, (x,y), xy, 1)
                xy[0] = -1
                pos2 = get_next_pos(graph, (x,y), xy, 1)
            else: #xy[1] == 1
                xy[0] = 1
                xy[1] = 0
                pos1 = get_next_pos(graph, (x,y), xy, -1)
                xy[0] = -1
                pos2 = get_next_pos(graph, (x,y), xy, -1)
                
            ret_pos = []
            for p in pos1:
                if p is not None:
                    ret_pos.append(p)
            for p in pos2:
                if p is not None:
                    ret_pos.append(p)
            if len(ret_pos) > 0:
                return ret_pos
            return [None]
    
        #J.  .L  
        #.F  7.

        #Diagonals
        if graph[x,y] == 'J' and graph[x+1,y+1] == 'F' and\
           graph[x,y+1] == '.' and xy[0] == -1:
            return [(x,y+1)]
        elif graph[x,y] == 'F' and graph[x-1,y-1] == 'J' and\
             graph[x,y-1] == '.' and xy[0] == 1:
            return [(x,y-1)]
        
        elif graph[x,y] == 'L' and graph[x+1,y-1] == '7' and\
             graph[x,y-1] == '.' and xy[0] == -1:
            return [(x,y-1)]
        elif graph[x,y] == '7' and graph[x-1,y+1] == 'L' and\
             graph[x,y+1] == '.' and xy[0] == 1:
            return [(x,y+1)]
        
        elif graph[x,y] == 'J' and graph[x+1,y+1] == 'F' and\
           graph[x+1,y] == '.' and xy[1] == -1:
            return [(x+1,y)]
        elif graph[x,y] == 'F' and graph[x-1,y-1] == 'J' and\
             graph[x-1,y] == '.' and xy[1] == 1:
            return [(x-1,y)]
        
        elif graph[x,y] == 'L' and graph[x+1,y-1] == '7' and\
             graph[x+1,y] == '.' and xy[1] == 1:
            return [(x+1,y)]
        elif graph[x,y] == '7' and graph[x-1,y+1] == 'L' and\
             graph[x-1,y] == '.' and xy[1] == -1:
            return [(x-1,y)]

        # 7 F
        # | | F--7
        # | | L--J
        # J L

        if xy[0] == -1:
            #left side up
            if graph[x,y] == 'J':
                if graph[x,y+1] != '.':
                    x += xy[0]
                    os = 1
                else:
                    return [(x,y+1)]
            
            elif graph[x,y] == '7' and os == 1:
                if graph[x,y+1] != '.' and graph[x-1,y] != '.':
                    xy[0] = 0
                    xy[1] = -1
                    y += xy[1]
                    os = -1
                elif graph[x,y+1] == '.':
                    return [(x,y+1)]
                else:
                    return [(x-1,y)]
            elif graph[x,y] == '7' and os == -1:
                xy[0] = 0
                xy[1] = -1
                y += xy[1]
                os = 1
                
            #right side up
            elif graph[x,y] == 'L':
                if graph[x,y-1] != '.':
                    x += xy[0]
                    os = -1
                else:
                    return [(x,y-1)]
                
            elif graph[x,y] == 'F' and os == -1:
                if graph[x,y-1] != '.' and graph[x-1,y] != '.':
                    xy[0] = 0
                    xy[1] = 1
                    y += xy[1]
                elif graph[x,y-1] == '.':
                    return [(x,y-1)]
                else:
                    return [(x-1,y)]
            elif graph[x,y] == 'F' and os == 1:
                xy[0] = 0
                xy[1] = 1
                y += xy[1]
            
            elif graph[x,y] == '|':
                if graph[x,y+os] != '.':
                    x += xy[0]
                else:
                    return [(x,y+os)]
           
            else:
                return [None]

        elif xy[0] == 1:
            #left side down
            if graph[x,y] == '7':
                if graph[x,y+1] != '.':
                    x += xy[0]
                    os = 1
                else:
                    return [(x,y+1)]
            
            elif graph[x,y] == 'J' and os == 1:
                if graph[x,y+1] != '.' and graph[x+1,y] != '.':
                    xy[0] = 0
                    xy[1] = -1
                    y += xy[1]
                elif graph[x,y+1] == '.':
                    return [(x,y+1)]
                else:
                    return [(x+1,y)]
            elif graph[x,y] == 'J' and os == -1:
                xy[0] = 0
                xy[1] = -1
                y += xy[1]
                
            #right side down
            elif graph[x,y] == 'F':
                if graph[x,y-1] != '.':
                    x += xy[0]
                    os = -1
                else:
                    return [(x,y-1)]
                
            elif graph[x,y] == 'L' and os == -1:
                if graph[x,y-1] != '.' and graph[x+1,y] != '.':
                    xy[0] = 0
                    xy[1] = 1
                    y += xy[1]
                    os = 1
                elif graph[x,y-1] == '.':
                    return [(x,y-1)]
                else:
                    return [(x+1,y)]
            elif graph[x,y] == 'L' and os == 1:
                xy[0] = 0
                xy[1] = 1
                y += xy[1]
                os = -1
            
            elif graph[x,y] == '|':
                if graph[x,y+os] != '.':
                    x += xy[0]
                else:
                    return [(x,y+os)]
           
            else:
                return [None]

        # L--J
        # F--7
        elif xy[1] == -1:
            #top side left
            if graph[x,y] == 'J':
                if graph[x+1,y] != '.':
                    y += xy[1]
                    os = 1
                else:
                    return [(x,y+1)]
            
            elif graph[x,y] == 'L' and os == 1:
                if graph[x+1,y] != '.' and graph[x,y-1] != '.':
                    xy[0] = -1
                    xy[1] = 0
                    x += xy[0]
                    os = -1
                elif graph[x+1,y] == '.':
                    return [(x+1,y)]
                else:
                    return [(x,y-1)]
            elif graph[x,y] == 'L' and os == -1:
                xy[0] = -1
                xy[1] = 0
                x += xy[0]
                os = 1
                
            #bottom side left
            elif graph[x,y] == '7':
                if graph[x-1,y] != '.':
                    y += xy[1]
                    os = -1
                else:
                    return [(x-1,y)]
                
            elif graph[x,y] == 'F' and os == -1:
                if graph[x-1,y] != '.' and graph[x,y-1] != '.':
                    xy[0] = 1
                    xy[1] = 0
                    x += xy[0]
                elif graph[x-1,y] == '.':
                    return [(x-1,y)]
                else:
                    return [(x,y-1)]
            elif graph[x,y] == 'F' and os == 1:
                xy[0] = 1
                xy[1] = 0
                x += xy[0]
            
            elif graph[x,y] == '-':
                if graph[x+os,y] != '.':
                    y += xy[1]
                else:
                    return [(x+os,y)]
           
            else:
                return [None]
            
        # L--J
        # F--7
        elif xy[1] == 1:
            #top side right
            if graph[x,y] == 'L':
                if graph[x+1,y] != '.':
                    y += xy[1]
                    os = 1
                else:
                    return [(x+1,y)]
            
            elif graph[x,y] == 'J' and os == 1:
                if graph[x+1,y] != '.' and graph[x,y+1] != '.':
                    xy[0] = -1
                    xy[1] = 0
                    x += xy[0]
                    os = 1
                elif graph[x+1,y] == '.':
                    return [(x+1,y)]
                else:
                    return [(x,y+1)]
            elif graph[x,y] == 'J' and os == -1:
                xy[0] = -1
                xy[1] = 0
                x += xy[0]
                
            #bottom side left
            elif graph[x,y] == 'F':
                if graph[x-1,y] != '.':
                    y += xy[1]
                    os = -1
                else:
                    return [(x-1,y)]
                
            elif graph[x,y] == '7' and os == -1:
                if graph[x-1,y] != '.' and graph[x,y+1] != '.':
                    xy[0] = 1
                    xy[1] = 0
                    x += xy[0]
                    os = 1
                elif graph[x-1,y] == '.':
                    return [(x-1,y)]
                else:
                    return [(x,y+1)]
            elif graph[x,y] == '7' and os == 1:
                xy[0] = 1
                xy[1] = 0
                x += xy[0]
                os = -1
            
            elif graph[x,y] == '-':
                if graph[x+os,y] != '.':
                    y += xy[1]
                else:
                    return [(x+os,y)]
           
            else:
                return [None]

In [6]:
#Breadth-First Search
def BFS_part2(graph, start):
    dxdy = [[-1,0], [1,0], [0,-1], [0,1]]
    queue = deque([start])
    dist = {start: 0}
    
    outgraph = deepcopy(graph)
    outgraph[start[0],start[1]] = 'O'
    
    while len(queue):
        cur_pos = queue.popleft()
        cur_dist = dist[cur_pos]
        cur_val = graph[cur_pos[0]][cur_pos[1]]

        for xy in dxdy:
            nxt_dist = cur_dist + 1
            nxt_poss = get_next_pos(graph, cur_pos, xy)
            
            for nxt_pos in nxt_poss:
                if nxt_pos is not None and nxt_pos not in dist.keys():
                    queue.append(nxt_pos)
                    dist[nxt_pos] = nxt_dist
                    outgraph[nxt_pos[0],nxt_pos[1]] = 'O'
        
    graph_size = graph.shape[0]*graph.shape[1]
    path = np.where(graph != '.')
    path_size = len(path[0])
    outside = len(dist.keys())
    
    inside = graph_size - path_size - outside
    
    return inside

graph2, start2 = get_graph(test2)
dist2, ngraph2 = BFS_part1(graph2, start2)
print(BFS_part2(ngraph2, (0,0)))

graph3, start3 = get_graph(test3)
dist3, ngraph3 = BFS_part1(graph3, start3)
print(BFS_part2(ngraph3, (0,0)))

graph4, start4 = get_graph(test4)
dist4, ngraph4 = BFS_part1(graph4, start4)
print(BFS_part2(ngraph4, (0,0)))

graph5, start5 = get_graph(test5)
dist5, ngraph5 = BFS_part1(graph5, start5)
print(BFS_part2(ngraph5, (0,0)))

print('Part 2 result:', BFS_part2(data_ngraph, (0,0)))

4
4
8
10
Part 2 result: 325


In [7]:
def ray_inside(graph, up=True):
    if up:
        edges = ['|', 'J', 'L']
    else:
        edges = ['|', '7', 'F']
    count = 0
    for line in graph:
        inside = False
        for c in line:
            if c == '.' and inside:
                count += 1
            elif c in edges:
                inside = not inside
    return count

print(ray_inside(ngraph2))
print(ray_inside(ngraph3))
print(ray_inside(ngraph4))
print(ray_inside(ngraph5))
print('Part 2 result:', ray_inside(data_ngraph))

4
4
8
10
Part 2 result: 325
